In [1]:
print 'hello world'

hello world


In [2]:
class MyContext(object):
    def __init__(self, *args):
        pass

In [3]:
class MyContext(object):
    def __init__(self, *args):
        self.data = args
        
    def __enter__(self):
        print "__enter__"
        return self.data
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        if exc_type:
            print "Exception:", exc_val
            
        print "__exit__"
        return  True

In [1]:
from contextlib import contextmanager

In [5]:
@contextmanager
def closing(o):
    print "enter"
    yield o
    print "exit"
    o.close()

In [7]:
with closing(open("/Users/zhoulei/PycharmProjects/learning/赠送python笔记第二版/1.弱引用.ipynb")) as f:
    print f.readline()

enter
{

exit


In [4]:
with MyContext(2,3,4) as data:
    print data.data

__enter__
Exception: 'tuple' object has no attribute 'data'
__exit__


# 以下为装饰器



In [5]:
def check_args(func):
    print 'hellow'
    def wrap(*args):
        args = filter(bool, args)
        func(*args)
    return wrap

In [7]:
@check_args
def test(*args):
    print args

hellow


In [8]:
test(123123,232)

(123123, 232)


In [8]:
class CheckArgs(object):
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        
        return self.func(*args, **kwargs)

In [9]:
@CheckArgs
def test(*args):
    print args

In [10]:
test(2222,222)

(2222, 222)


In [27]:
def singleton(cls):
    class wrap(cls):
        def __new__(cls, *args, **kwargs):
            o = getattr(cls, '_instance',None)
            if not o:
                o= super(wrap, cls).__new__(cls,*args, **kwargs)
                cls._instance = o
            return o
    return wrap

In [28]:
@singleton
class A(object):
    def test(self):print hex(id(self))

In [29]:
a, b =A(), A()

In [30]:
a is b

True

In [31]:
def action(cls):
    cls.mvc = staticmethod(lambda :"Action")
    return cls

In [32]:
@action
class Login(object):pass

In [33]:
Login.mvc()

'Action'

In [47]:
class MyDeacriptor(object):
    def __get__(self, instance, owner):
        print "get", instance, owner
        return hex(id(instance))
    
    def __set__(self, instance, value):
        print "set", instance, value
        
    def __delete__(self, instance):
        print "del:" , instance
class Data(object):
    x = MyDeacriptor()

In [37]:
d = Data()

In [38]:
d.x

get <__main__.Data object at 0x10f7675d0> <class '__main__.Data'>


'0x10f7675d0'

In [39]:
d.x =100

set <__main__.Data object at 0x10f7675d0> 100


In [40]:
d.x

get <__main__.Data object at 0x10f7675d0> <class '__main__.Data'>


'0x10f7675d0'

In [41]:
del d.x

del: <__main__.Data object at 0x10f7675d0>


In [42]:
Data.x

get None <class '__main__.Data'>


'0x10dda3678'

In [43]:
d.x

get <__main__.Data object at 0x10f7675d0> <class '__main__.Data'>


'0x10f7675d0'

In [44]:
type(d.x)

get <__main__.Data object at 0x10f7675d0> <class '__main__.Data'>


str

In [1]:
class MyDeacriptor(object):
    def __get__(self, instance, owner):
        print "get", instance, owner
        return hex(id(instance))
    
    # def __set__(self, instance, value):
    #     print "set", instance, value
    #     
    # def __delete__(self, instance):
    #     print "del:" , instance
        
class Data(object):
    x = MyDeacriptor()

In [58]:
a  = Data()
a.x

get <__main__.Data object at 0x10f761190> <class '__main__.Data'>


'0x10f761190'

In [59]:
a.x = 100

In [8]:
class Data(object):
    pass

print Data.__class__
print type.__class__

<type 'type'>
<type 'type'>


In [9]:
Data.__metaclass__

AttributeError: type object 'Data' has no attribute '__metaclass__'

In [10]:
class InjectMeta(type):
    def __new__(cls, *args, **kwargs):
        t = type.__new__(cls,*args, **kwargs)
        def print_id(self):print hex(id(self))
        t.print_id = print_id
        t.s = "hellworld!"
        return t

In [11]:
class Data(object):
    __metaclass__ = InjectMeta

In [12]:
Data.__metaclass__

__main__.InjectMeta

In [13]:
Data.__class__

__main__.InjectMeta

In [14]:
dir(Data)

['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__metaclass__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'print_id',
 's']

In [15]:
Data.s

'hellworld!'

In [16]:
Data().print_id()

0x109e78110


In [18]:
class InjectMeta(type):
    def __new__(cls, name, bases,attrs):
        print "class" ,cls
        print "name" , name
        print "bases", bases
        print "attrs:", attrs
        return type.__new__(cls, name, bases, attrs)
    
    def __init__(cls, name, bases, attrs):
        print "class" , cls
        type.__init__(cls,name,bases, attrs)

In [19]:
class Data(object):
    __metaclass__ = InjectMeta
    x = 1
    def test(self):pass

class <class '__main__.InjectMeta'>
name Data
bases (<type 'object'>,)
attrs: {'test': <function test at 0x109e6faa0>, 'x': 1, '__module__': '__main__', '__metaclass__': <class '__main__.InjectMeta'>}
class <class '__main__.Data'>
